#### 공사비 1차 전처리

##### 레코드 제거
* 계약전력: 100 이상
* 접수종류명: 신설(상용/임시) 이외
* 총공사비: 60,000,000이상(11개 레코드로 예외처리)
* 공사형태명: '외선소요' 이외(나머지는 140개 정도로 예외처리)

##### 컬럼 제거
* 컬럼값이 1개로 구성된 컬럼은 학습에 영향이 없기 때문에 제거

In [2]:
import re
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
from freeman.plt_setting import plt_settings
from freeman.aiddd.data_manager import read_data, write_data

# 한글처리 지원
plt_settings()

#### 원본 데이터 불러오기

In [3]:
_start = datetime.now()
df_data = read_data('2nd provide cons')
print(
    f'Total Elapsed Time for Data Load: {datetime.now()-_start}\n'
    f'Shape: {df_data.shape}'
)

Total Elapsed Time for Data Load: 0:00:27.156337
Shape: (19052, 143)


#### 데이터 전처리

##### 결측치 처리

In [4]:
# 결측치 값의 상태 체크
df_data.isna().sum().sort_values(ascending=False)[:5]

신기술공종규격코드    19052
전선변압기코드번호    19052
CUD상태코드      19052
공사번호             0
더블블레이드수익금        0
dtype: int64

* 모든 레코드가 결측치인 컬럼이 3개 -> 0으로 치환해 나중에 단일값 컬럼 제거시 같이 제거

In [5]:
# '신기술공종규격코드', '전선변압기코드번호', '상태코드' 3컬럼이 모두 결측치이기 때문에,
# 단일값인 0으로 치환 후 이후 단일값으로 구성된 컬럼 삭제시 제거
df_data.fillna(0, inplace=True)

##### 학습대상 레코드 추출

In [6]:
# 학습대상 레코드 조건
training_data_conditions = \
    (df_data['접수종류명'] == '신설(상용/임시)') & \
    (df_data['계약전력'] < 100) & \
    (df_data['공사형태코드'] == 2) & \
    (df_data['총공사비'] < 60000000)
    
# 학습대상 이외의 데이터 건수 확인
df_data[~training_data_conditions].shape

(3158, 143)

In [7]:
# 전체 레코드(19,052)중에 3,158건으로 예외로 제거
# (제거 후 인덱스를 갱신함)
df_data = df_data[training_data_conditions].reset_index(drop=True)

##### 1개의 값으로 구성된 컬럼 제거

In [8]:
nunique_per_column = df_data.nunique()
single_value_column_list = nunique_per_column[nunique_per_column == 1].index
df_data.drop(columns=single_value_column_list, axis=1, inplace=True)

# 결과확인
print(
    f'Number of Drop Columns: {len(single_value_column_list)}\n'
    f'Columns: {[col for col in single_value_column_list]}\n'
    f'Data shape: {df_data.shape}'
)

Number of Drop Columns: 89
Columns: ['공사변경순번', '배전공사형태코드', '외자분재료비', '사업소재료비금액', '간접재료비', '작업부산물값', '전선변압기코드번호', '전선변압기추가금액', '전선변압기조정금액', '인력분기준비용', '기계분기준비용', '할선분기준비용', '전선기술노무비', '전선기술재료비', '전선기술경비', '무정전기술노무비', '무정전기술경비', '도급분무정전기술노무비', '도급분무정전기술경비', '감리총공사비', '폐기물총공사비', '자동화총공사비', '도통시험총공사비', '전주가지지공법노무비', '전주가지지공법경비', '공통공사비', '가설공사비1', '가설공사비2', '공동장비비용', '현장관리비', '직접공사비', '회사분직접공사비', '도급분직접공사비', '간접공사비', '가공분단위시설별공사비', '지중분단위시설별공사비', '가공분회사분재료비', '지중분회사분재료비', '가공분도급분재료비', '지중분도급분재료비', '감리자본적금액', '감리수익적금액', '폐기자본적금액', '폐기수익적금액', '도통자본적금액', '도통수익적금액', '자동화자본적금액', '자동화수익적금액', '자본대비자본적비율', '수익대비수익적비율', '원형전주버팀목공법자재비', '원형전주버팀목공법노무비', '원형전주버팀목공법경비', '도급분원형전주버팀목공법자재비', '도급분원형전주버팀목공법노무비', '도급분원형전주버팀목공법경비', '더블블레이드총공사비', '더블블레이드자본금액', '더블블레이드수익금', '사급자재자체감리비', '고객부담자체감리비', '부대공사자체감리비', '신기술공종규격코드', '신기술공사비', '신기술적용대수', '신기술적용선로길이', '중공스크루로드총공사비', '중공스크루로드자본적금액', '중공스크루로드수익적금액', 'CUD상태코드', '손실보상금', '손실보상계수', '무정전전선이선공사비', '무정전전선이선특허사용료율', '방호대총공사비', '방호대자본적금액', '방호대수익적금액', '미관개선총공사비', 

#### 주요 컬럼 데이터 분석

##### 공사번호

In [9]:
# 공사번호에 영문 포함 여부 확인
df_data['공사번호'][df_data['공사번호'].str.contains('[a-zA-Z]')].shape

(2189,)

* 공사번호에 영문이 포함된 레코드가 2,189개 존재
* 공사번호 자체를 학습에 사용하지 않기 때문에 Key로서 보관

##### 날짜 데이터
* `최종등록일시`, `최종변경일시`

In [10]:
df_data[['최초등록일시', '최종변경일시']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15894 entries, 0 to 15893
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   최초등록일시  15894 non-null  datetime64[ns]
 1   최종변경일시  15894 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 248.5 KB


In [11]:
df_data[['최초등록일시', '최종변경일시']].head(2)

,최초등록일시,최종변경일시
0,2021-06-03 08:21:39,2021-06-03 08:21:40
1,2021-06-16 16:59:17,2021-06-16 16:59:17


In [12]:
# 최초등록일시와 최종변경일시가 각각 의미가 있는가를 판단하기 위해
# 두 일시의 차이 계산(하루 미만이면 하나(최종변경일시)만 학습에 사용)
SECONDS_PER_HOUR = 60 * 60
difference_date_condition = (
    (df_data['최종변경일시']-df_data['최초등록일시']).dt.total_seconds() >= SECONDS_PER_HOUR
)

df_data[difference_date_condition].shape

(0, 54)

* 두 날짜 데이터의 차이가 1시간 이상인 데이터가 하나도 없기 때문에 두 데이터를 유사 데이터로 인식하고 `최종변경일시`만 학습에 사용함

##### 날짜 관련 컬럼 추가

In [13]:
df_data['year'] = df_data['최종변경일시'].dt.year
df_data['month'] = df_data['최종변경일시'].dt.month
df_data['day'] = df_data['최종변경일시'].dt.day
df_data['dayofweek'] = df_data['최종변경일시'].dt.dayofweek
df_data['dayofyear'] = df_data['최종변경일시'].dt.dayofyear

##### 사번 
* `최초등록자사번` vs `최종변경자사번`

In [14]:
difference_employee_number_condition = \
    (df_data['최초등록자사번'] != df_data['최종변경자사번'])

df_data[difference_employee_number_condition].shape

(0, 59)

* 두 사번 데이터가 모두 동일하기 때문에 `최종변경자사번`만 학습에 사용

##### 사번 관련 컬럼 추가

In [ ]:
df_data['사번코드'] = df_data['최종변경자사번'].str.extract('([a-zA-Z]+)', expand=False)
df_data['사번코드'].value_counts()